## Explore K-Means clustering usando R e princípios de dados organizados.

### [**Questionário pré-aula**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

Nesta lição, vais aprender a criar clusters utilizando o pacote Tidymodels e outros pacotes do ecossistema R (vamos chamá-los de amigos 🧑‍🤝‍🧑), e o conjunto de dados de música nigeriana que importaste anteriormente. Vamos abordar os fundamentos do K-Means para Clustering. Lembra-te de que, como aprendeste na lição anterior, existem várias formas de trabalhar com clusters e o método que utilizas depende dos teus dados. Vamos experimentar o K-Means, pois é a técnica de clustering mais comum. Vamos começar!

Termos que vais aprender:

-   Pontuação de Silhueta

-   Método do Cotovelo

-   Inércia

-   Variância

### **Introdução**

[K-Means Clustering](https://wikipedia.org/wiki/K-means_clustering) é um método derivado do domínio do processamento de sinais. É utilizado para dividir e agrupar conjuntos de dados em `k clusters` com base em semelhanças nas suas características.

Os clusters podem ser visualizados como [Diagramas de Voronoi](https://wikipedia.org/wiki/Voronoi_diagram), que incluem um ponto (ou 'semente') e a sua região correspondente.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infográfico por Jen Looper</figcaption>


O clustering K-Means segue os seguintes passos:

1.  O cientista de dados começa por especificar o número desejado de clusters a serem criados.

2.  Em seguida, o algoritmo seleciona aleatoriamente K observações do conjunto de dados para servir como os centros iniciais dos clusters (ou seja, os centróides).

3.  Depois, cada uma das observações restantes é atribuída ao centróide mais próximo.

4.  A seguir, a nova média de cada cluster é calculada e o centróide é movido para essa média.

5.  Agora que os centros foram recalculados, cada observação é novamente verificada para ver se pode estar mais próxima de um cluster diferente. Todos os objetos são novamente reatribuídos utilizando as médias atualizadas dos clusters. Os passos de atribuição de clusters e atualização dos centróides são repetidos iterativamente até que as atribuições de clusters deixem de mudar (ou seja, quando se alcança a convergência). Normalmente, o algoritmo termina quando cada nova iteração resulta em um movimento insignificante dos centróides e os clusters tornam-se estáticos.

<div>

> Nota que, devido à randomização das observações iniciais k utilizadas como centróides iniciais, podemos obter resultados ligeiramente diferentes cada vez que aplicamos o procedimento. Por esta razão, a maioria dos algoritmos utiliza vários *inícios aleatórios* e escolhe a iteração com o menor WCSS. Assim, é fortemente recomendado executar o K-Means com vários valores de *nstart* para evitar um *ótimo local indesejável.*

</div>

Esta curta animação utilizando a [arte](https://github.com/allisonhorst/stats-illustrations) de Allison Horst explica o processo de clustering:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Arte por @allison_horst</figcaption>



Uma questão fundamental que surge no clustering é esta: como sabes quantos clusters deves separar nos teus dados? Uma desvantagem do uso do K-Means é o facto de que precisas de estabelecer `k`, ou seja, o número de `centróides`. Felizmente, o `método do cotovelo` ajuda a estimar um bom valor inicial para `k`. Vais experimentá-lo em breve.

### 

**Pré-requisito**

Vamos começar exatamente de onde parámos na [lição anterior](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), onde analisámos o conjunto de dados, fizemos várias visualizações e filtrámos o conjunto de dados para observações de interesse. Certifica-te de que o revisitas!

Vamos precisar de alguns pacotes para completar este módulo. Podes instalá-los com: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternativamente, o script abaixo verifica se tens os pacotes necessários para completar este módulo e instala-os para ti caso alguns estejam em falta.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Vamos começar com tudo!

## 1. Uma dança com dados: Reduzir aos 3 géneros musicais mais populares

Este é um resumo do que fizemos na lição anterior. Vamos explorar e analisar alguns dados!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Isso correu bem!

## 2. Mais exploração de dados.

Quão limpos estão estes dados? Vamos verificar a existência de valores atípicos utilizando boxplots. Vamos concentrar-nos em colunas numéricas com menos valores atípicos (embora seja possível limpar os valores atípicos). Os boxplots podem mostrar o intervalo dos dados e ajudar a escolher quais colunas usar. Note que os boxplots não mostram a variância, um elemento importante para dados bem agrupáveis. Por favor, veja [esta discussão](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) para leitura adicional.

[Boxplots](https://en.wikipedia.org/wiki/Box_plot) são usados para representar graficamente a distribuição de dados `numéricos`, então vamos começar por *selecionar* todas as colunas numéricas juntamente com os géneros musicais populares.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Veja como o auxiliar de seleção `where` torna isso fácil 💁? Explore outras funções semelhantes [aqui](https://tidyselect.r-lib.org/).

Como vamos criar um boxplot para cada característica numérica e queremos evitar o uso de loops, vamos reformular os nossos dados para um formato *mais longo* que nos permitirá tirar proveito de `facets` - subgráficos que exibem cada subconjunto dos dados.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Muito mais longo! Agora é hora de alguns `ggplots`! Então, que `geom` vamos usar?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Agora podemos ver que estes dados estão um pouco ruidosos: ao observar cada coluna como um boxplot, é possível identificar valores atípicos. Poderíamos percorrer o conjunto de dados e remover esses valores atípicos, mas isso tornaria os dados bastante reduzidos.

Por agora, vamos escolher quais colunas utilizaremos para o nosso exercício de clustering. Vamos selecionar as colunas numéricas com intervalos semelhantes. Poderíamos codificar o `artist_top_genre` como numérico, mas vamos descartá-lo por enquanto.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Computação de clustering k-means em R

Podemos calcular k-means em R utilizando a função integrada `kmeans`, veja `help("kmeans()")`. A função `kmeans()` aceita um data frame com todas as colunas numéricas como seu argumento principal.

O primeiro passo ao usar clustering k-means é especificar o número de clusters (k) que serão gerados na solução final. Sabemos que existem 3 géneros musicais que extraímos do conjunto de dados, então vamos tentar com 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


O objeto kmeans contém várias informações que estão bem explicadas em `help("kmeans()")`. Por agora, vamos concentrar-nos em algumas. Vemos que os dados foram agrupados em 3 clusters com tamanhos de 65, 110, 111. O resultado também contém os centros dos clusters (médias) para os 3 grupos ao longo das 5 variáveis.

O vetor de clustering é a atribuição de cluster para cada observação. Vamos usar a função `augment` para adicionar a atribuição de cluster ao conjunto de dados original.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfeito, acabámos de dividir o nosso conjunto de dados em 3 grupos. Então, quão bom é o nosso agrupamento 🤷? Vamos dar uma olhada no `Silhouette score`.

### **Silhouette score**

[A análise de Silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) pode ser usada para estudar a distância de separação entre os clusters resultantes. Este score varia de -1 a 1, e se o score estiver próximo de 1, o cluster é denso e bem separado dos outros clusters. Um valor próximo de 0 representa clusters sobrepostos com amostras muito próximas da fronteira de decisão dos clusters vizinhos. [fonte](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

O método de silhouette médio calcula a média do silhouette das observações para diferentes valores de *k*. Um score médio de silhouette elevado indica um bom agrupamento.

A função `silhouette` no pacote de cluster é usada para calcular a largura média do silhouette.

> O silhouette pode ser calculado com qualquer [métrica de distância](https://en.wikipedia.org/wiki/Distance "Distance"), como a [distância Euclidiana](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") ou a [distância Manhattan](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), que discutimos na [lição anterior](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


A nossa pontuação é **0,549**, ou seja, bem no meio. Isto indica que os nossos dados não são particularmente adequados para este tipo de agrupamento. Vamos ver se conseguimos confirmar esta suspeita visualmente. O [pacote factoextra](https://rpkgs.datanovia.com/factoextra/index.html) fornece funções (`fviz_cluster()`) para visualizar agrupamentos.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


A sobreposição nos clusters indica que os nossos dados não são particularmente adequados para este tipo de clustering, mas vamos continuar.

## 4. Determinar o número ideal de clusters

Uma questão fundamental que frequentemente surge no clustering K-Means é esta: sem etiquetas de classe conhecidas, como saber quantos clusters usar para separar os seus dados?

Uma forma de tentar descobrir é usar uma amostra de dados para `criar uma série de modelos de clustering` com um número crescente de clusters (por exemplo, de 1 a 10) e avaliar métricas de clustering como o **Silhouette score.**

Vamos determinar o número ideal de clusters calculando o algoritmo de clustering para diferentes valores de *k* e avaliando o **Within Cluster Sum of Squares** (WCSS). O total de soma de quadrados dentro do cluster (WCSS) mede a compacidade do clustering, e queremos que seja o menor possível, com valores mais baixos significando que os pontos de dados estão mais próximos.

Vamos explorar o efeito de diferentes escolhas de `k`, de 1 a 10, neste clustering.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Agora que temos o total da soma dos quadrados dentro dos clusters (tot.withinss) para cada algoritmo de clustering com centro *k*, utilizamos o [método do cotovelo](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) para encontrar o número ideal de clusters. O método consiste em traçar o WCSS como uma função do número de clusters e escolher o [cotovelo da curva](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Cotovelo da curva") como o número de clusters a ser utilizado.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


O gráfico mostra uma grande redução no WCSS (ou seja, maior *coerência*) à medida que o número de clusters aumenta de um para dois, e uma redução adicional notável de dois para três clusters. Depois disso, a redução torna-se menos acentuada, resultando num `cotovelo` 💪 no gráfico por volta de três clusters. Isto é uma boa indicação de que existem dois a três clusters de pontos de dados razoavelmente bem separados.

Podemos agora avançar e extrair o modelo de clustering onde `k = 3`:

> `pull()`: usado para extrair uma única coluna
>
> `pluck()`: usado para indexar estruturas de dados como listas


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Ótimo! Vamos visualizar os clusters obtidos. Que tal adicionar um pouco de interatividade usando `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Talvez esperássemos que cada grupo (representado por cores diferentes) tivesse géneros distintos (representados por formas diferentes).

Vamos analisar a precisão do modelo.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


A precisão deste modelo não é má, mas também não é excelente. Pode ser que os dados não sejam adequados para o K-Means Clustering. Estes dados são demasiado desequilibrados, pouco correlacionados e há muita variância entre os valores das colunas para formar clusters eficazes. Na verdade, os clusters que se formam provavelmente são fortemente influenciados ou enviesados pelas três categorias de género que definimos acima.

Ainda assim, foi um processo de aprendizagem interessante!

Na documentação do Scikit-learn, pode-se ver que um modelo como este, com clusters pouco bem definidos, tem um problema de 'variância':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infográfico do Scikit-learn</figcaption>



## **Variância**

A variância é definida como "a média das diferenças quadradas em relação à média" [fonte](https://www.mathsisfun.com/data/standard-deviation.html). No contexto deste problema de clustering, refere-se aos dados em que os números do nosso conjunto tendem a divergir um pouco demais da média.

✅ Este é um ótimo momento para pensar em todas as formas de corrigir este problema. Ajustar os dados um pouco mais? Usar colunas diferentes? Utilizar um algoritmo diferente? Dica: Experimente [escalar os seus dados](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) para normalizá-los e testar outras colunas.

> Experimente este '[calculador de variância](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' para compreender melhor o conceito.

------------------------------------------------------------------------

## **🚀Desafio**

Dedique algum tempo a este notebook, ajustando os parâmetros. Consegue melhorar a precisão do modelo ao limpar mais os dados (removendo outliers, por exemplo)? Pode usar pesos para dar mais importância a determinadas amostras de dados. O que mais pode fazer para criar clusters melhores?

Dica: Experimente escalar os seus dados. Há código comentado no notebook que adiciona escalonamento padrão para fazer com que as colunas de dados se assemelhem mais em termos de intervalo. Vai perceber que, embora o score de silhueta diminua, o 'cotovelo' no gráfico suaviza-se. Isto acontece porque deixar os dados sem escala permite que dados com menos variância tenham mais peso. Leia mais sobre este problema [aqui](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Questionário pós-aula**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Revisão & Autoestudo**

-   Dê uma olhada num Simulador de K-Means [como este](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Pode usar esta ferramenta para visualizar pontos de dados de amostra e determinar os seus centróides. Pode editar a aleatoriedade dos dados, o número de clusters e o número de centróides. Isto ajuda-o a ter uma ideia de como os dados podem ser agrupados?

-   Além disso, veja [este documento sobre K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) da Stanford.

Quer testar as suas novas competências de clustering em conjuntos de dados que se adaptam bem ao K-Means clustering? Veja:

-   [Treinar e Avaliar Modelos de Clustering](https://rpubs.com/eR_ic/clustering) usando Tidymodels e amigos

-   [Análise de Cluster K-Means](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [Clustering K-Means com princípios de dados organizados](https://www.tidymodels.org/learn/statistics/k-means/)

## **Tarefa**

[Experimente diferentes métodos de clustering](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## AGRADECIMENTOS A:

[Jen Looper](https://www.twitter.com/jenlooper) por criar a versão original em Python deste módulo ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) por criar as ilustrações incríveis que tornam o R mais acolhedor e envolvente. Encontre mais ilustrações na sua [galeria](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Boas aprendizagens,

[Eric](https://twitter.com/ericntay), Embaixador Estudante Gold da Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Arte por @allison_horst</figcaption>



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante notar que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
